In [ ]:
#Installing libraries
!pip3 install --upgrade tensorflow-gpu
!pip3 install tensorflow-hub

In [ ]:
# Importing libraries
from keras.utils.np_utils import to_categorical

import pandas as pd
import numpy as np

import re

import sklearn
from sklearn.feature_extraction.text import TfidfVectorizer
from itertools import combinations
from sklearn.cluster import KMeans

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

import tensorflow as tf
import tensorflow.compat.v1 as tf
import tensorflow_hub as hub
import numpy as np

import matplotlib.pyplot as plt
import nltk

nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')
nltk.download('stopwords')

nltk.download('punkt')

import spacy 
from spacy import displacy

import random

# from rake_nltk import Rake

from numpy import array
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten,Embedding,Dense

import os
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorboard.plugins import projector

from gensim.scripts.glove2word2vec import glove2word2vec

# import en_core_web_lg
# nlp = en_core_web_lg.load()

from nltk.tokenize import word_tokenize

import gensim
from scipy import spatial

import csv
import tensorflow as tf

# import transformers
# transformers.logging.set_verbosity(transformers.logging.CRITICAL)

# from sentence_transformers import SentenceTransformer, util
# import nmslib

# distilbert = SentenceTransformer('distilbert-base-uncased')

import multiprocessing
import math

import time
from gensim.models import KeyedVectors
from gensim.models.fasttext import FastText
import pandas as pd
import gensim
gensim.__version__

In [ ]:
#Importing dataset
data = pd.read_csv("/content/drive/MyDrive/MScA - UChicago/Assignments/Quarter 3 - Spring 2022/Capstone Project/ml-25m/movie_all_info.csv")
movtag3 = pd.read_pickle("/content/drive/MyDrive/MScA - UChicago/Assignments/Quarter 3 - Spring 2022/Capstone Project/Final datasets/2_ner_tags.pkl")

In [ ]:
#Using tensorflow universal sentence encoder on tags
%%time
# movtag3 = movtag2.copy()

module_url = "https://tfhub.dev/google/universal-sentence-encoder/4" 
model = hub.load(module_url)
print ("module %s loaded" % module_url)

movtag3['tags'] = movtag3['tags'].apply(lambda x: ' '.join(x))
movtag3['tag_people'] = movtag3['tag_people'].apply(lambda x: ' '.join(x))
movtag3['tag_org'] = movtag3['tag_org'].apply(lambda x: ' '.join(x))
movtag3['tag_norp'] = movtag3['tag_norp'].apply(lambda x: ' '.join(x))
movtag3['tag_location'] = movtag3['tag_location'].apply(lambda x: ' '.join(x))

movtag3['tags_vector'] = movtag3['tags'].apply(lambda x: model([x]).numpy())
movtag3['tags_people_vector'] = movtag3['tag_people'].apply(lambda x: model([x]).numpy())
movtag3['tags_org_vector'] = movtag3['tag_org'].apply(lambda x: model([x]).numpy())
movtag3['tags_norp_vector'] = movtag3['tag_norp'].apply(lambda x: model([x]).numpy())
movtag3['tags_location_vector'] = movtag3['tag_location'].apply(lambda x: model([x]).numpy())

movtag3.head()

## Tensorboard Projections

In [ ]:
# Set up a logs directory, so Tensorboard knows where to look for files.
log_dir='/content/drive/MyDrive/MScA - UChicago/Assignments/Quarter 3 - Spring 2022/Capstone Project/Final datasets/Tensorboard/3_TUSE/'
if not os.path.exists(log_dir):
    os.makedirs(log_dir)


titles = []
weights = np.vstack((movtag3['tags_vector'][0], movtag3['tags_vector'][1]))
titles.append(data['title'][0])
titles.append(data['title'][1])
for i in range(2,len(movtag3['tags_vector'])):
  try: 
    math.isnan(movtag3['tags_vector'][i])
  except:
    weights = np.vstack((weights, movtag3['tags_vector'][i]))
    titles.append(data['title'][i])

# Save Labels separately on a line-by-line manner.
with open(os.path.join(log_dir, 'metadata.tsv'), "w") as f:
  for subwords in titles:
    f.write("{}\n".format(subwords))
  # Fill in the rest of the labels with "unknown".
  # for unknown in range(1, encoder.vocab_size - len(encoder.subwords)):
  #   f.write("unknown #{}\n".format(unknown))


# Save the weights we want to analyze as a variable. Note that the first
# value represents any unknown word, which is not in the metadata, here
# we will remove this value.
weights = tf.Variable(weights)
# Create a checkpoint from embedding, the filename and key are the
# name of the tensor.
checkpoint = tf.train.Checkpoint(embedding=weights)
checkpoint.save(os.path.join(log_dir, "embedding.ckpt"))

# Set up config.
config = projector.ProjectorConfig()
embedding = config.embeddings.add()
# The name of the tensor will be suffixed by `/.ATTRIBUTES/VARIABLE_VALUE`.
embedding.tensor_name = "embedding/.ATTRIBUTES/VARIABLE_VALUE"
embedding.metadata_path = 'metadata.tsv'
projector.visualize_embeddings(log_dir, config)

In [ ]:
# Now run tensorboard against on log data we just saved.
%load_ext tensorboard
%tensorboard --logdir /content/drive/MyDrive/3_TUSE/

In [ ]:
movtag3.to_pickle("/content/drive/MyDrive/MScA - UChicago/Assignments/Quarter 3 - Spring 2022/Capstone Project/Final datasets/3_TUSE_tags.pkl")